In [36]:
%run constants.ipynb
%run read_sheet.ipynb

In [18]:
# Load the arguments
with open(ARGUMENTS_FILENAME, "r") as fid:
    arguments = json.load(fid)

In [21]:
df = read_sheet(arguments["sheet_id"], col_range="A:ZZ")

### Pivot, clean up dates

In [22]:
df.columns = ["metric"] + df.columns.tolist()[1:]
df.set_index("metric", inplace=True)
df = df.T
df.reset_index(inplace=True)
df.columns = df.columns.tolist()
df.rename(columns={"index": "date"}, inplace=True)

In [23]:
def _parse_date(date_str: str) -> datetime.datetime:
    """
        Dates are formatted like 'Sun 11/2'.
        If it's a date range like '11/2 - 11/4', it selects the latest date
    """

    date_str = date_str.split(" ")[-1]
    # months and days are not zero padded
    month = int(date_str.split("/")[0])
    day = int(date_str.split("/")[1])
    
    # if no year present, assume 2023
    if len(date_str.split("/")) == 3:
        # year would be '23' or '24'
        year = 2000 + int(date_str.split("/")[2])
    else:
        year = 2023
        
    return datetime.datetime(year, month, day)

df["date clean"] = df["date"].map(_parse_date)
df.set_index("date clean", inplace=True)
df.sort_index(inplace=True)

### Medication

In [26]:
# remove headers and blank column names
bad_cols = [
    np.nan, "MEDICATION", "SYMPTOMS", "MENSTRUAL", np.nan, "Daily in morning",
]

bad_cols = [c for c in bad_cols if c in df.columns]
df.drop(columns=bad_cols, axis=1, inplace=True)

In [27]:
med_times = [c for c in df.columns if ":00" in c]

def get_daily_dosage(row, drug, unit):
    """"
        row: row of df
        drug: string of drug name
        unit: ml, mg
        
        "500 Ibuprofen, 1000 mg Acetaminophen" -> 1000
    """
    total_dose = 0
    for med_time in med_times:
        observation = str(row[med_time])
        mentions = re.findall(rf"(\d+\.?\d+) {unit} {drug}", observation)
        if mentions:
            dose = float(mentions[0])
            total_dose += dose
    return total_dose


def get_med_count(row: dict, value: str):
    total = 0
    for med_time in med_times:
        observation = str(row[med_time])
        if value in observation.lower():
            total += 1
    return total

In [28]:
unit = "mg"
mg_drugs = [
    "Acetaminophen",
    "Ibuprofen",
    "Duloxetine",
    "Naproxen",
    "THC",
    "CBD",  # ignore CBN and CBG for now
]
for drug in mg_drugs:
    df[f"{drug} {unit}"] = df.apply(get_daily_dosage, axis=1, args=(drug, unit))


unit = "ml"
ml_drugs = [
    "THC/CBD",
    "CBD oil",
]
for drug in ml_drugs:
    df[f"{drug} {unit}"] = df.apply(get_daily_dosage, axis=1, args=(drug, unit))


df["protab_pills"] = df.apply(get_med_count, axis=1, args=("protab",))

    
# Calculate mg metrics
# Assume 0.25 ml of 1:1 THC/CBD is equivalent to 3.75 mg THC and 3.75 mg CBD
df["THC mg"] = df["THC mg"] + ((3.75 / 0.25) * df["THC/CBD ml"])
df["THC mg"] = df["THC mg"] + (2 * df["protab_pills"])
df["CBD mg"] = df["CBD mg"] + ((3.75 / 0.25) * df["THC/CBD ml"])
df["CBD mg"] += ((3.75 / 0.25) * df["CBD oil ml"])

#### Make numeric scores

In [29]:
for pain_metric in [c for c in df.columns if "pain" in c]:
    df[f"{pain_metric} numeric"] = pd.to_numeric(df[pain_metric], errors="coerce")


# Pain inferred from pain reported + meds
df["pelvic_pain_inferred"] = df["pelvic pain numeric"]

# Add a point for every 1500 mg Acetaminophen / day and every 4 mg THC
df["pelvic_pain_inferred"] += df["Acetaminophen mg"] // 1500
df["pelvic_pain_inferred"] += df["THC mg"] // 4

In [30]:
def quantify_headache(headache_str: str) -> int:
    values = {
        "yes": 1,
        "extreme": 2,
    }
    return values.get(str(headache_str), 0)


df["headache_numeric"] = df["headache"].map(quantify_headache)


def quantify_fatigue(fatigue_str: str) -> int:
    values = {
        "yes": 1,
        "extreme": 2,
        "ok": 0,
    }
    return values.get(str(fatigue_str), 0)


df["fatigue_numeric"] = df["fatigue"].map(quantify_fatigue)
df["day of cycle"] = pd.to_numeric(df["day of cycle"])

In [31]:
# which cycle is this?
first_days = df.loc[df["day of cycle"] == 1].index.values

def get_nth_cycle(row):
    if np.isnan(row["day of cycle"]):
        return np.nan
    previous_first_days = [
        d for d in first_days
        if d <= row["date clean"]
    ]
    return len(previous_first_days)

df["nth_cycle"] = df.reset_index().apply(get_nth_cycle, axis=1).tolist()

In [32]:
df["peak_day"] = df["cycle event"] == "peak day"
df.loc[df["cycle event"].isnull(), "peak_day"] = np.nan

# the last day of the period is a day of bleeding followed by no bleeding
bleeding_symbols = ["L", "M", "H", "S", "B"]

df["symbol_tomorrow"] = df["symbol"].shift(-1)

df["last_bleeding_day"] = (
    df["symbol"].isin(bleeding_symbols) &
    (~df["symbol_tomorrow"].isin(bleeding_symbols)) &
    (df["day of cycle"] < 10)
    # make sure we're not including random days of spotting in the middle of the cycle
)

# last cycle day is the day before the next nth_cycle, aka before day of cycle is 1
df["day_of_cycle_tomorrow"] = df["day of cycle"].shift(-1)
df["last_cycle_day"] = (df["day_of_cycle_tomorrow"] == 1)

In [33]:
# marker for whether we fully charted this cycle
# this will exclude cycles where we omitted symbols
# but doesn't exclude the last cycle, where we haven't reached the end yet

subset = df[df["day of cycle"].notnull()]
subset["day of cycle"] = subset["day of cycle"].astype(int)
subset.loc[subset["symbol"] == "", "symbol"] = np.nan

table = subset.pivot(
    index="nth_cycle",
    columns="day of cycle",
    values="symbol",
)

day_count = pd.DataFrame(subset["nth_cycle"].value_counts())
symbol_count = pd.DataFrame(table.notnull().sum(axis=1))

merged = day_count.merge(
    symbol_count,
    left_index=True,
    right_index=True,
)
merged.columns = ["days_count", "symbols_count"]
valid_cycles = merged.loc[merged["symbols_count"] == merged["days_count"]].index.values

# valid_cycles = table[table.notnull().sum(axis=1) > 0].index.values
df["valid_cycle"] = df["nth_cycle"].isin(valid_cycles)

In [34]:
df.to_csv(arguments["dataframe_outfile"], index=True)